In [ ]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql (2).json


In [ ]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)

In [ ]:
import random
random.seed(2021)

### Products 테이블

In [ ]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

0

In [ ]:
kitchen_goods = ['스푼','접시','와인잔','컵','병','캔', 
                 '도시락통','젓가락','포크','나이프']
bathroom_goods = ['수건','수건걸이','쓰레기통','휴지걸이','변기솔',
                  '선반','거울','샤워기','샤워커튼','샤워볼']
furniture_goods = ['책상','의자','침대','소파','수납장','옷장',
                   '어린이가구','식탁','장식장','선반']
category_names = ['주방용품', '욕실용품','가구']

In [ ]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([kitchen_goods, bathroom_goods, furniture_goods]):
  for item in item_list:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    cur.execute(sql,(item,price,category_names[i],cost))

conn.commit()

### Sales 테이블

In [ ]:
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

0

In [ ]:
company_names= ['한셈', '리바트', '일룸', '오늘의집', '굿인테리어']


In [ ]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}

In [ ]:
sql = "insert into sales values (default,%s,%s,%s,%s);"
for month, day in calendar.items():
  for i in range(10):
    date= f'2020-{month}-{random.randint(1,day)}'
    company = company_names[random.randint(0,4)]
    pid = random.randint(11,40)
    unit=random.randint(1,20)
    cur.execute(sql, (date, company, pid, unit))
conn.commit()

### 두 테이블을 Join한 View 생성

In [ ]:
sql = """
  CREATE VIEW sales_books as 
    SELECT sid, sdate, scompany, pid, pname, pcategory,
    pprice, sunit, pcost, pprice*sunit AS Revenue, (pprice-pcost)*sunit AS profit
    From sales
    Join products
    ON sales.spid = products.pid
    ORDER BY sdate;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

OperationalError: ignored

* 데이터를 sales_book.csv 로 저장

In [ ]:
sql = 'select * from sales_books'
cur.execute(sql)
results = cur.fetchall()

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,350,2020-01-01,오늘의집,12,접시,주방용품,44700,16,41570,715200,50080
1,110,2020-01-01,한셈,34,소파,가구,27100,7,23850,189700,22750
2,104,2020-01-02,굿인테리어,12,접시,주방용품,44700,4,41570,178800,12520
3,342,2020-01-02,한셈,14,컵,주방용품,87100,6,77520,522600,57480
4,108,2020-01-03,일룸,12,접시,주방용품,44700,13,41570,581100,40690


In [ ]:
del df[3]
del df[0]

In [ ]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,오늘의집,접시,주방용품,44700,16,41570,715200,50080
1,2020-01-01,한셈,소파,가구,27100,7,23850,189700,22750
2,2020-01-02,굿인테리어,접시,주방용품,44700,4,41570,178800,12520
3,2020-01-02,한셈,컵,주방용품,87100,6,77520,522600,57480
4,2020-01-03,일룸,접시,주방용품,44700,13,41570,581100,40690


In [ ]:
df.to_csv('sales_books.csv', index=False)

### 마무리

In [ ]:
cur.close()
conn.close()